In [ ]:
from collections import defaultdict
import random

class Student:
    def __init__(self, name, register_number, preferred_day, institute_id):
        self.name = name
        self.register_number = register_number
        self.preferred_day = preferred_day
        self.assigned_day = None
        self.institute_id = institute_id

class Institution:
    def __init__(self, institute_id, name, total_students):
        self.institute_id = institute_id
        self.name = name
        self.total_students = total_students
        self.registered_students = []

    def register_student(self, student):
        if len(self.registered_students) < self.total_students:
            self.registered_students.append(student)
        else:
            print(f"{self.name} has reached its limit of {self.total_students} students.")

    def get_eligible_students(self):
        eligible_count = max(1, len(self.registered_students) // 2)
        return random.sample(self.registered_students, eligible_count)

class University:
    def __init__(self, slots_per_day):
        self.slots_per_day = slots_per_day
        self.institutes = {}
        self.day_slots = {
            'Day1': {'max_capacity': slots_per_day, 'current_bookings': 0, 'students': []},
            'Day2': {'max_capacity': slots_per_day, 'current_bookings': 0, 'students': []},
            'Day3': {'max_capacity': slots_per_day, 'current_bookings': 0, 'students': []}
        }
        self.bookings_by_institution = defaultdict(lambda: {'Day1': [], 'Day2': [], 'Day3': []})

    def add_institution(self, institute_id, name, total_students):
        self.institutes[institute_id] = Institution(institute_id, name, total_students)

    def register_student(self, institute_id, name, register_number, preferred_day):
        if institute_id in self.institutes:
            student = Student(name, register_number, preferred_day, institute_id)
            self.institutes[institute_id].register_student(student)
        else:
            print(f"Institution ID {institute_id} not found.")

    def book_slots(self):
        for institute in self.institutes.values():
            eligible_students = institute.get_eligible_students()
            for student in eligible_students:
                if not self._try_book_student(student):
                    print(f"No available slots for {student.name} on preferred or alternate days.")

    def _try_book_student(self, student):
        preferred_day = student.preferred_day
        if self._book_day(preferred_day, student):
            return True
        for day in self.day_slots:
            if day != preferred_day and self._book_day(day, student):
                return True
        return False

    def _book_day(self, day, student):
        slot = self.day_slots[day]
        if slot['current_bookings'] < slot['max_capacity']:
            student.assigned_day = day
            slot['students'].append(student)
            slot['current_bookings'] += 1
            self.bookings_by_institution[student.institute_id][day].append(student)
            return True
        return False

    def daywise_student_count(self):
        result = {day: defaultdict(int) for day in self.day_slots}
        for day, slot in self.day_slots.items():
            for student in slot['students']:
                institute_name = self.institutes[student.institute_id].name
                result[day][institute_name] += 1
        print("\nDaywise student count for each institute:")
        for day, counts in result.items():
            print(f"{day}: {dict(counts)}")

    def list_students_by_day_and_institution(self, day, institution_id):
        if day not in self.day_slots or institution_id not in self.institutes:
            print("Invalid day or institution ID.")
            return []
        institute_students = [
            {"name": student.name, "register_number": student.register_number}
            for student in self.day_slots[day]['students']
            if student.institute_id == institution_id
        ]
        institute_name = self.institutes[institution_id].name
        print(f"\nStudents from {institute_name} on {day}: {institute_students}")
        return institute_students


university = University(slots_per_day=100)

university.add_institution(1, "Institute A", 20)
university.add_institution(2, "Institute B", 15)
university.add_institution(3, "Institute C", 10)

university.register_student(1, "Alice", "A001", "Day1")
university.register_student(1, "Bob", "A002", "Day2")
university.register_student(1, "Charlie", "A003", "Day1")
university.register_student(2, "David", "B001", "Day1")
university.register_student(2, "Eve", "B002", "Day2")
university.register_student(2, "Frank", "B003", "Day2")
university.register_student(3, "Grace", "C001", "Day1")
university.register_student(3, "Hank", "C002", "Day3")
university.register_student(3, "Ivy", "C003", "Day3")

university.book_slots()

university.daywise_student_count()

university.list_students_by_day_and_institution("Day1", 1)
university.list_students_by_day_and_institution("Day2", 2)
university.list_students_by_day_and_institution("Day3", 3)
